# Laboratorio 5: Optimización Multiobjetivo

**Por:** Amalia Carbonell & Santiago Casasbuenas

## Problema 1: Optimización Multiobjetivo en Distribución de Recursos para Misión Humanitaria

### 1. Formulación del Modelo Multiobjetivo

#### a. Recursos

$R$: Conjunto de recursos = {alimentos, medicinas, equipos médicos, agua, mantas}

$A$: Conjunto de aviones = {1,2,3,4}

$Z$: Conjunto de zonas de destino = {A,B,C,D}

$V$: Conjunto de viajes por avión = {1,2} (Cada avión puede hacer hasta 2 viajes)

#### b. Parámetros

**Recursos:**

- $v_i$: valor de impacto social por tonelada del recurso $i$

- $p_i$: Peso por unidad del recurso $i$

- $vol_i$: Volumen por unidad del recurso $i$

- $disp_i$: Unidades disponibles del recurso $i$

**Aviones:**

- $capp_j$: Capacidad en peso del avión $j$ (TON)

- $capvol_j$: Capacidad en volumen del avión $j$ $(m^3)$

- $cfijo_j$: Costo fijo de usar avión $j$ (miles USD)

- $cvar_j$: Costo variable por km del avión $j$ (miles USD/km)

**Zonas:**

- $d_k$: Distancia a la zona k (km)

- $m_k$: Multiplicador de impacto social de la zona $k$

- $min_{ik}$: Necesidad mínima del recurso $i$ en la zona $k$

#### c. Variables de Decisión

- $x_{ijvk}$: Cantidad (TON) del recursos $i$ transportado por avión $j$ en el viaje $v$ a la zona $k$

- $y_{jvk} \in$ {0,1}: 1 si el avión $j$ en el viaje $v$ va a la zona $k$

- $v_j \in$ {0,1}: 1 si el avión $j$ es usado en al menos de un viaje, 0 si no.

- $n \_ equipos_{jvk}$: Cantidad entera de unidades de equipos transportados (por indivisibilidad)

#### d. Funciones Objetivo

**Objetivo 1**: Maximizar el valor de impacto social total

$$
Z_1 = \sum_{i \in R} \sum_{j \in A} \sum_{v \in V} \sum_{k \in Z} v_i * x_{ijvk} * m_k
$$

**Objetivo 2**: Minimizar el costo total del transporte

$$
Z_2 = \sum_{j \in A} cfijo_j * u_j + \sum_{j \in A} \sum_{v \in V} \sum_{k \in Z} cvar_j * d_k * y_{jvk}
$$


#### e. Restricciones

**Restricción 1**: capacidad de peso de cada avión por viaje

Cada avión tiene un límite de peso por viaje que no debe ser excedido.

$$
\sum_{i \in R} p_i * x_{ijvk} \leq capp_j \quad \forall j \in A, \quad \forall v \in V, \quad \forall k \in Z
$$

**Restricción 2**: Capacidad de volumen de cada avión por viaje

Cada avión tambien tiene un límite de volumen.

$$
\sum_{i \in R} vol_i * x_{ijvk} \leq capvol_j \quad \forall j \in A, \quad \forall v \in V, \quad \forall k \in Z
$$




**Restricción 3:** Seguridad de medicamentos

Los medicamentos no pueden ser transportados en el avión 1, por falta de refrigeración.

$$
x_{medicinas, 1, v, k} = 0 \quad \forall v \in V, \quad \forall k \in Z 
$$

**Restricción 4:** Incompatibilidad entre agua potable y equipos médicos

estos dos recursos no pueden ser transportados juntos en el mismo viaje, por riesgo de contaminación.

$$
x_{agua, 1, v, k} > 0 ⇒ x_{equipos, j, v, k} \quad \forall j \in A \quad \forall v \in V, \quad \forall k \in Z 
$$

Esta restricción toca expresarla con variables binarias en la modelación de piomo, ya que también funciona en el sentido inverso, pero esto es una aproximación de lo que representa la restricción y su funcionamiento.

**Restricción 5:** Indivisibilidad de equipos mmédicos

Cada equipo medico pesa 0.3 TON y debe enviarse en unidades completas no separables.

$$
x_{equipos, j, v, k} = 0.3 * n_{jvk}, \quad n_{jvk} \in \mathbb{Z}_{\leq 0} \quad \forall j,v,k
$$

**Restrección 6: Zona única por viaje**

Un avión en un viaje solo puede ir a una zona:

$$
\sum_{k \in Z} y_{jvk} \leq 1 \quad \forall j \in A, \forall v \in V
$$

Y para que los $x_{ijvk}$ tengan sentido, se deben enviar recursos solo si el avión fue asignado a esa zona:

$$
x_{ijvk} \le M * y_jvk \quad \forall i, j, v, k
$$

donde $M$ es un valor lo suficientemente grande.

**Restricción 7:** Satisfacción de necesidades mínimas

Cada zona debe recibir al menos la cantidad mínima requerida de cada recurso:

$$
\sum_{j \in A} \sum_{v \in V} \leq min_{ik} \quad \forall i \in R, \forall k \in Z
$$

**Restricción 8:** Disponibilidad máxima de recursos

No se puede transportar más de lo que hay disponible:

$$
\sum_{j \in A} \sum_{v \in V} \sum_{k \in Z} \frac{x_{ijvk}}{p_i} \leq disp_i \quad \forall i \in R
$$

donde $x$ se trabaja en toneladas y la disponibilidad está en unidades.

**Restricción 9**: Activación del avión si se usa en algún viaje

Si un avión se usa en al menos un viaje, entonces su variable binaria debe activarse:

$$
y_{jik} \leq u_j \quad \forall j, v, k
$$

### 2. Implementación del modelo en pyomo

Definición de Modelo Base en Pyomo

In [ ]:
from pyomo.environ import *

model = ConcreteModel()

# CONJUNTOS

# Recursos
model.R = Set(initialize = ['alimentos', 'medicinas', 'equipos', 'agua', 'mantas'])

# Aviones
model.A = Set(initialize = [1,2,3,4])

# Zonas
model.Z = Set(initialize = ['A', 'B', 'C', 'D'])

# Viajes por avión
model.V = Set(initialize = [1,2])

Parámetros

In [ ]:
# PARÁMETROS

# Valor de impacto por tonelada
valor_impacto = {
    'alimentos': 50, 
    'medicinas': 100, 
    'equipos': 120, 
    'agua': 60, 
    'mantas': 40
}
model.valor_impacto = Param(model.R, initialize=valor_impacto)

# Peso por unidad (TON/unidad)
peso = {
    'alimentos': 5,
    'medicinas': 2,
    'equipos': 0.3,
    'agua': 6,
    'mantas': 3
}
model.peso = Param(model.R, initialize=peso)

# volumen por unidad
volumen = {
    'alimentos': 3,
    'medicinas': 1,
    'equipos': 0.5,
    'agua': 4,
    'mantas': 2
}
model.volumen = Param(model.R, initialize=volumen)

# disponibilidad en unidades
disponibilidad = {
    'alimentos': 12,
    'medicinas': 15,
    'equipos': 40,
    'agua': 15,
    'mantas': 20
}
model.disponibilidad = Param(model.R, initialize=disponibilidad)

# capacidad de peso y volumen de los aviones
cap_peso = {
    1: 40,
    2: 50,
    3: 60,
    4: 45
}
model.cap_peso = Param(model.A, initialize=cap_peso)

# capacidad de volumen de los aviones
cap_vol = {
    1: 35,
    2: 40,
    3: 45,
    4: 38
}
model.cap_vol = Param(model.A, initialize=cap_vol)

# Costo fijo de cada avión 
costo_fijo = {
    1: 15,
    2: 20,
    3: 25,
    4: 18
}
model.costo_fijo = Param(model.A, initialize=costo_fijo)

# Costo variable por km
costo_variable = {
    1: 0.020,
    2: 0.025,
    3: 0.030,
    4: 0.022
}
model.costo_variable = Param(model.A, initialize=costo_variable)

# Distancia a cada zona
distancia = {
    'A': 800,
    'B': 1200,
    'C': 1500,
    'D': 900
}
model.distancia = Param(model.Z, initialize=distancia)

# Multiplicador de impacto social de cada zona
multiplicador = {
    'A': 1.2,
    'B': 1.5,
    'C': 1.8,
    'D': 1.4
}
model.multiplicador = Param(model.Z, initialize=multiplicador)

# Necesidades mínimas de cada recurso por zona
necesidades_minimas = {
    ('alimentos', 'A'): 8,  ('agua', 'A'): 6,   ('medicinas', 'A'): 2,  ('equipos', 'A'): 0.6,  ('mantas', 'A'): 3,
    ('alimentos', 'B'): 12, ('agua', 'B'): 9,   ('medicinas', 'B'): 3,  ('equipos', 'B'): 0.9,  ('mantas', 'B'): 5,
    ('alimentos', 'C'): 16, ('agua', 'C'): 12,  ('medicinas', 'C'): 4,  ('equipos', 'C'): 1.2,  ('mantas', 'C'): 7,
    ('alimentos', 'D'): 10, ('agua', 'D'): 8,   ('medicinas', 'D'): 2,  ('equipos', 'D'): 0.6,  ('mantas', 'D'): 4
}
model.min_necesidad = Param(model.R, model.Z, initialize=necesidades_minimas)


Variables

In [ ]:
# Cantidad de recursos i enviados por avión j en viaje v a k
model.x = Var(model.R, model.A, model.V, model.Z, within=NonNegativeReals)

# Binaria y: 1 si el avión j en el viaje v va a la zona k, 0 en caso contrario
model.y = Var(model.A, model.V, model.Z, within=Binary)

# Bnaria u: 1 si el avión j es utilizado
model.u = Var(model.A, within=Binary)

# Número de equipos médicos transportados (indivisible)
model.n_equipos = Var(model.A, model.V, model.Z, within=NonNegativeIntegers) 

# Variables auxiliares
model.presencia_agua = Var(model.A, model.V, model.Z, within=Binary)
model.presencia_equipos = Var(model.A, model.V, model.Z, within=Binary)

Restricciones

In [ ]:
# RESTRICCIONES

# Capacidad de peso por viaje
def capacidad_peso_rule(model, j, v, k):
    return sum(model.peso[i] * model.x[i,j,v,k] for i in model.R) <= model.cap_peso[j]

model.capacidad_peso = Constraint(model.A, model.V, model.Z, rule=capacidad_peso_rule)


# Capacidad de volumen por viaje
def capacidad_vol_rule(model, j, v, k):
    return sum(model.volumen[i] * model.x[i,j,v,k] for i in model.R) <= model.cap_vol[j]

model.CapacidadVolumen = Constraint(model.A, model.V, model.Z, rule=capacidad_vol_rule)


# Medicina no pueden ir en el avión 1 
def restriccion_medicinas_avion1_rule(model, v, k):
    return model.x['medicinas', 1, v, k] == 0

model.RestriccionMedicinas = Constraint(model.V, model.Z, rule=restriccion_medicinas_avion1_rule)


# Solo una zona por viaje
def unica_zona_por_viaje_rule(model, j, v):
    return sum(model.y[j, v, k] for k in model.Z) <= 1

model.ZonaUnicaPorViaje = Constraint(model.A, model.V, rule=unica_zona_por_viaje_rule)


M = 9999999

# Si se transporta agua, entoncees presencia_agua = 1
def agua_implica_presencia_rule(model, j, v, k):
    return model.x['agua', j, v, k] <= M * model.presencia_agua[j, v, k]

model.AguaPresente = Constraint(model.A, model.V, model.Z, rule=agua_implica_presencia_rule)


# Si se transporta equipos, entonces presencia_equipos = 1
def equipos_implica_presencia_rule(model, j, v, k):
    return model.x['equipos', j, v, k] <= M * model.presencia_equipos[j, v, k]

model.EquiposPresentes = Constraint(model.A, model.V, model.Z, rule=equipos_implica_presencia_rule)


# El agua y los equipos no pueden estar juntos
def no_agua_y_equipos_juntos_rule(model, j, v, k):
    return model.presencia_agua[j, v, k] + model.presencia_equipos[j, v, k] <= 1

model.CompatibilidadAguaEquipos = Constraint(model.A, model.V, model.Z, rule=no_agua_y_equipos_juntos_rule)


# Indivisibilidad de equipos médicos
def indivisibilidad_rule(model, j, v, k):
    return model.x['equipos', j, v, k] == 0.3 * model.n_equipos[j, v, k]

model.IndivisibilidadEquipos = Constraint(model.A, model.V, model.Z, rule=indivisibilidad_rule)


def envio_si_asignado_rule(model, i, j, v, k):
    return model.x[i, j, v, k] <= M * model.y[j, v, k]

model.EnvioSoloSiAsignado = Constraint(model.R, model.A, model.V, model.Z, rule=envio_si_asignado_rule)


# Satisfacción de necesidades mínimas
def necesidades_minimas_rule(model, i, k):
    return sum(model.x[i, j, v, k] for j in model.A for v in model.V) >= model.min_necesidad[i, k]

model.NecesidadesMinimas = Constraint(model.R, model.Z, rule=necesidades_minimas_rule)


# No exceder disponibilidad
def disponibilidad_maxima_rule(model, i):
    total_transportado = sum(model.x[i, j, v, k] / model.peso[i] for j in model.A for v in model.V for k in model.Z)
    return total_transportado <= model.disponibilidad[i]

model.LimiteDisponibilidad = Constraint(model.R, rule=disponibilidad_maxima_rule)


# Activación de avión si se usa
def uso_avion_rule(model, j, v, k):
    return model.y[j, v, k] <= model.u[j]

model.UsoAvion = Constraint(model.A, model.V, model.Z, rule=uso_avion_rule)


# Activar el avión si transporta algo (ligar u[j] con x[i,j,v,k])
def uso_avion_implicado_por_envio_rule(model, j):
    return sum(model.x[i, j, v, k] for i in model.R for v in model.V for k in model.Z) <= M * model.u[j]

model.ActivarAvionSiTransporta = Constraint(model.A, rule=uso_avion_implicado_por_envio_rule)


#Activar un viaje si hay carga para una zona (ligar y[j,v,k] con x[i,j,v,k])
def activar_y_si_envio_rule(model, j, v, k):
    return sum(model.x[i, j, v, k] for i in model.R) <= M * model.y[j, v, k]

model.ActivarViajeSiEnvio = Constraint(model.A, model.V, model.Z, rule=activar_y_si_envio_rule)


Funciones Objetivo

Maximizar el Valor de Impacto Social ($Z_1$)


$$
Z_1 = \sum_{i \in R} \sum_{j \in A} \sum_{v \in V} \sum_{k \in Z} v_i * x_{ijvk} * m_k
$$


In [ ]:
def impacto_social_rule(model):
    return sum(model.valor_impacto[i] * model.x[i, j, v, k] * model.multiplicador[k]
               for i in model.R for j in model.A for v in model.V for k in model.Z)
model.ImpactoSocial = Objective(rule=impacto_social_rule, sense=maximize)

Minimizar el costo total de transporte ($Z_2$)

$$
Z_2 = \sum{j \in A} cfijo_j * u_j + \sum_{j \in A} \sum_{v \in V} \sum_{k \in Z} cvar_j * d_k * y_{jvk}
$$

In [ ]:
"""

def costo_total_rule(model):
    return sum(model.costo_fijo[j] * model.u[j] for j in model.A) + \
           sum(model.costo_variable[j] * model.distancia[k] * model.y[j, v, k]
               for j in model.A for v in model.V for k in model.Z)
model.CostoTotal = Objective(rule=costo_total_rule, sense=minimize)

"""

### 3. Implementación del Método ϵ-constrain

**¿Por qué ϵ-constrain?**

Los objetivos que tenemos son muy distintos entre sí. Por un lado, queremos maximizar el impacto social y por el otro queremos minimizar. Queremos asegurarque las necesidades mínimas se cumplen primero, luego ver cómo hacer el transporte "barato". Es muy probable que el frente de Pareto no sea perfectamente convexo, por la rigidez de las necesidades mínimas y la indivisibilidad de euqipos médicos.

El método ϵ-constrain nos asegura un alto impacto social, limitando el costo de una manera contralada mediante ϵ. Finalmente, nos permite capturar soluciones que la suma ponderada podría pasar por alto (en zonas no convexas del frente).

**Plan:**
Maximizar el objetivo principal (Impacto social) y convertir el segundo (costo total) en una restricción con distintos valores de ϵ.

1. Mantenemos el impacto social como única función objetivo (done)

2. Agregar restricción de costo total $\leq$ ϵ

In [ ]:
model.epsilon = Param(initialize=9999, mutable=True)  # valor inicial alto, mutable para poder cambiarlo

3. El costo total queda como restricción

In [ ]:
def costo_total_expr(model):
    return sum(model.costo_fijo[j] * model.u[j] for j in model.A) + \
           sum(model.costo_variable[j] * model.distancia[k] * model.y[j, v, k]
               for j in model.A for v in model.V for k in model.Z)

model.CostoTotalExpr = Expression(rule=costo_total_expr)

4. Agregar restricción de tipo ϵ

In [ ]:
def epsilon_constraint_rule(model):
    return model.CostoTotalExpr <= model.epsilon

model.EpsilonRestriccionCosto = Constraint(rule=epsilon_constraint_rule)

5 Resolver el modelo con distintos valores de ϵ

In [ ]:
model.ImpactoSocial.activate()

from pyomo.opt import SolverFactory, TerminationCondition
from pyomo.environ import value
import matplotlib.pyplot as plt

# Crear solver
solver = SolverFactory('glpk')

# Lista de valores de ϵ a evaluar
epsilons = list(range(420, 550, 10))  # puedes ajustar el rango

# Lista para guardar las soluciones (costo, impacto)
pareto_solutions = []

# Resolver para cada valor de ϵ
for eps in epsilons:
    model.epsilon = eps
    results = solver.solve(model, tee=False)
    
    if (results.solver.termination_condition == TerminationCondition.optimal or
        results.solver.termination_condition == TerminationCondition.feasible):
        impacto = value(model.ImpactoSocial)
        costo = value(model.CostoTotalExpr)
        pareto_solutions.append((costo, impacto))
        print(f"✔️ Épsilon = {eps} → Costo: {costo:.2f}, Impacto: {impacto:.2f}")
    else:
        print(f"❌ Épsilon = {eps} → No factible")

# Mostrar soluciones
print("\n📊 Soluciones de Pareto:")
for c, i in pareto_solutions:
    print(f" - Costo: {c:.2f}  |  Impacto: {i:.2f}")


6. Graficamos el frente de pareto

In [ ]:
# Extraer listas para graficar
costos, impactos = zip(*pareto_solutions)

plt.figure(figsize=(8, 6))
plt.plot(costos, impactos, marker='o', linestyle='-')
plt.xlabel('Costo total (miles USD)')
plt.ylabel('Impacto social')
plt.title('Frente de Pareto (Método ϵ-constraint)')
plt.grid(True)
plt.tight_layout()
plt.show()


### 4. Análisis y Discusión Adicional

## Problema 2: Optimización Multiobjetivo en PLanificación de Rutas de Inspección

### 1. Formulación del Modelo Multiobjetivo

### 2. Selección y Justificación del Método de Resolución

### 3. Implementación y Análisis

### 4. Análisis y Discusión Adicional